#### Load LLM and device

In [1]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(model="gemini-2.5-flash")

In [2]:
import torch

# Set the torch device
if torch.cuda.is_available():  # NVIDIA GPU
    device = torch.device("cuda")
elif torch.backends.mps.is_available():  # macOS
    device = torch.device("mps")
else:
    device = torch.device("cpu")  # CPU
print(f"Using {device.type} device")

Using cuda device


#### LongForm UQ

In [3]:
from uqlm import LongFormUQ

In [4]:
prompts = ["write a paragraph about Paul McCartney", "write a paragraph about John Lennon"]

In [5]:
# Sentence-Response UQ: aggregated response-level scoring
sent_lfuq = LongFormUQ(llm=llm, granularity="sentence", mode="unit_response", aggregation_method="mean", device=device)
result = await sent_lfuq.generate_and_score(prompts=prompts, num_responses=2)
result.to_df()

Output()

Output()

,response,sampled_responses,prompt,sentence_set,entailment,noncontradiction,contrasted_entailment
0,"Paul McCartney is an iconic British musician, ...",[Sir Paul McCartney stands as one of the most ...,write a paragraph about Paul McCartney,"[Paul McCartney is an iconic British musician,...",0.485355,0.998730,0.994352
1,John Lennon remains one of the most influentia...,[John Lennon remains one of the most iconic an...,write a paragraph about John Lennon,[John Lennon remains one of the most influenti...,0.864676,0.999407,0.999056


In [6]:
# Sentence-Response UQ: sentence-level scoring
sent_lfuq = LongFormUQ(llm=llm, granularity="sentence", mode="unit_response", aggregation_method=None, device=device)
result = await sent_lfuq.generate_and_score(prompts=prompts, num_responses=2)
result.to_df()

Output()

Output()

,response,sampled_responses,prompt,sentence_set,entailment,noncontradiction,contrasted_entailment
0,Sir Paul McCartney is one of the most celebrat...,[Sir Paul McCartney stands as one of the most ...,write a paragraph about Paul McCartney,[Sir Paul McCartney is one of the most celebra...,"[0.9667072296142578, 0.5901162475347519, 0.921...","[0.9994462132453918, 0.9991875886917114, 0.999...","[0.9994173645973206, 0.9969615638256073, 0.999..."
1,John Lennon remains one of the most iconic and...,"[John Lennon, an iconic figure in 20th-century...",write a paragraph about John Lennon,[John Lennon remains one of the most iconic an...,"[0.9902316629886627, 0.942951887845993, 0.9036...","[0.9996265769004822, 0.9995386898517609, 0.999...","[0.999622255563736, 0.9995062351226807, 0.9991..."


In [7]:
# Claim-Response UQ:  claim-level scoring
claim_lfuq = LongFormUQ(llm=llm, granularity="claim", mode="unit_response", aggregation_method=None, device=device)
result = await claim_lfuq.generate_and_score(prompts=prompts, num_responses=2)
result.to_df()

Output()

Output()

,response,sampled_responses,prompt,claim_set,entailment,noncontradiction,contrasted_entailment
0,Paul McCartney is an undisputed titan of music...,[Sir Paul McCartney stands as one of the most ...,write a paragraph about Paul McCartney,[Paul McCartney is an undisputed titan of musi...,"[0.8392975330352783, 0.4484418034553528, 0.064...","[0.9980470538139343, 0.9929606914520264, 0.986...","[0.9976764023303986, 0.9839319288730621, 0.702..."
1,John Lennon remains one of the most iconic and...,[John Lennon remains one of the most iconic an...,write a paragraph about John Lennon,[John Lennon is one of the most iconic figures...,"[0.9698392152786255, 0.9648253917694092, 0.553...","[0.9995621740818024, 0.9995166659355164, 0.678...","[0.9995458722114563, 0.9994959831237793, 0.574..."


In [5]:
# matched claim UQ: claim-level scoring
matched_claim_lfuq = LongFormUQ(llm=llm, granularity="claim", mode="matched_unit", aggregation_method=None, device=device)
result = await matched_claim_lfuq.generate_and_score(prompts=prompts, num_responses=2)
result.to_df()

Output()

Output()

,response,sampled_responses,prompt,claim_set,entailment,noncontradiction,contrasted_entailment,cosine_sim,bert_score
0,Sir Paul McCartney stands as one of the most i...,[Paul McCartney stands as one of the most cele...,write a paragraph about Paul McCartney,[Sir Paul McCartney is one of the most iconic ...,"[0.5723282769322395, 0.5403065569698811, 0.614...","[0.9995814561843872, 0.9996109306812286, 0.999...","[0.9970733225345612, 0.9955335855484009, 0.995...","[0.9795704931020737, 0.9917527288198471, 0.950...","[0.9822260737419128, 0.9859554171562195, 0.962..."
1,John Lennon remains one of the most iconic and...,[John Lennon remains one of the most iconic an...,write a paragraph about John Lennon,[John Lennon remains one of the most iconic fi...,"[0.5577000826597214, 0.5492749214172363, 0.563...","[0.9996176064014435, 0.9996269941329956, 0.998...","[0.9963062703609467, 0.9962500929832458, 0.995...","[0.9820920377969742, 0.9827326089143753, 0.956...","[0.983920693397522, 0.9822609722614288, 0.9589..."


In [5]:
# matched sentence UQ: sentence-level scoring
matched_sent_lfuq = LongFormUQ(llm=llm, granularity="sentence", mode="matched_unit", aggregation_method=None, device=device)
result = await matched_sent_lfuq.generate_and_score(prompts=prompts, num_responses=2)
result.to_df()

Output()

Output()

,response,sampled_responses,prompt,sentence_set,entailment,noncontradiction,contrasted_entailment,cosine_sim,bert_score
0,Paul McCartney stands as one of the most iconi...,[Sir Paul McCartney is one of the most iconic ...,write a paragraph about Paul McCartney,[Paul McCartney stands as one of the most icon...,"[0.8904413282871246, 0.05869482085108757, 0.82...","[0.9995529651641846, 0.9992298185825348, 0.999...","[0.9993205666542053, 0.9838801324367523, 0.999...","[0.9508362859487534, 0.8191811144351959, 0.957...","[0.9439810812473297, 0.8969708681106567, 0.925..."
1,John Lennon remains one of the most iconic and...,[John Lennon remains one of the most iconic an...,write a paragraph about John Lennon,[John Lennon remains one of the most iconic an...,"[0.12136035785079002, 0.2189687117934227, 0.21...","[0.9990199506282806, 0.9993440508842468, 0.999...","[0.9889121353626251, 0.9963703453540802, 0.992...","[0.957445427775383, 0.9118515104055405, 0.9181...","[0.952458381652832, 0.8947174251079559, 0.8968..."


#### Decomposition

In [6]:
from uqlm.longform import ResponseDecomposer

In [7]:
rd = ResponseDecomposer(llm)

In [8]:
responses = [
    "Hello there! Today I visited Dr. Chauhan at his home. He was hanging his framed B.S., M.S., and Ph.D. degrees. It was fun!", 
    "One time I went on a class trip to D.C. to visit the white house. While there, I felt in awe of the beutiful art and important people with titles such as Prof., Esq., and others."
]

sampled_responses = [
    [
        "Hi! I went to see Dr. Chauhan at his house today. He was busy putting up his framed B.S., M.S., and Ph.D. diplomas. It was quite enjoyable!", 
        "Greetings! Today, I stopped by Dr. Chauhan's home. He was in the process of displaying his framed degrees: B.S., M.S., and Ph.D. It was a delightful experience!"
    ],
    [
        "Once, I took a school trip to Washington, D.C. to see the White House. While I was there, I was amazed by the stunning artwork and the notable individuals with titles like Professor, Esquire, and more.",
        "During a class excursion to Washington, D.C., I had the opportunity to visit the White House. I was struck by the beautiful art and the distinguished people holding titles such as Prof., Esq., and others.",
    ],
]

In [9]:
# Decompose responses into sentences
rd.decompose_sentences(responses=responses)

[['Hello there!',
  'Today I visited Dr. Chauhan at his home.',
  'He was hanging his framed B.S., M.S., and Ph.D. degrees.',
  'It was fun!'],
 ['One time I went on a class trip to D.C. to visit the white house.',
  'While there, I felt in awe of the beutiful art and important people with titles such as Prof., Esq., and others.']]

In [10]:
# Decompose sampled responses into sentences
rd.decompose_candidate_sentences(sampled_responses=sampled_responses)

[[['Hi!',
   'I went to see Dr. Chauhan at his house today.',
   'He was busy putting up his framed B.S., M.S., and Ph.D. diplomas.',
   'It was quite enjoyable!'],
  ['Greetings!',
   "Today, I stopped by Dr. Chauhan's home.",
   'He was in the process of displaying his framed degrees: B.S., M.S., and Ph.D. It was a delightful experience!']],
 [['Once, I took a school trip to Washington, D.C. to see the White House.',
   'While I was there, I was amazed by the stunning artwork and the notable individuals with titles like Professor, Esquire, and more.'],
  ['During a class excursion to Washington, D.C., I had the opportunity to visit the White House.',
   'I was struck by the beautiful art and the distinguished people holding titles such as Prof., Esq., and others.']]]

In [12]:
# Decompose responses into claims
await rd.decompose_claims(responses=responses)

[['I visited Dr. Chauhan.\n',
  'I visited Dr. Chauhan at his home.\n',
  'Dr. Chauhan was hanging his degrees.\n',
  'Dr. Chauhan was hanging his B.S. degree.\n',
  'Dr. Chauhan was hanging his M.S. degree.\n',
  'Dr. Chauhan was hanging his Ph.D. degree.\n',
  "Dr. Chauhan's degrees were framed.\n",
  'It was fun.\n'],
 ['I went on a class trip.\n',
  'The class trip was to D.C.\n',
  'I visited the white house.\n',
  'I felt in awe.\n',
  'The art was beautiful.\n',
  'The people were important.\n',
  'The people had titles.\n',
  'The titles included Prof..\n',
  'The titles included Esq..\n',
  'The titles included others.\n\n']]

In [15]:
# Decompose sampled responses into claims
await rd.decompose_candidate_claims(sampled_responses=sampled_responses)

[[['I went to see Dr. Chauhan.\n',
   'I went to see Dr. Chauhan at his house.\n',
   'I went to see Dr. Chauhan today.\n',
   'Dr. Chauhan was busy.\n',
   'Dr. Chauhan was putting up diplomas.\n',
   'Dr. Chauhan was putting up framed diplomas.\n',
   "Dr. Chauhan's diplomas were framed.\n",
   "Dr. Chauhan's diplomas were B.S. diplomas.\n",
   "Dr. Chauhan's diplomas were M.S. diplomas.\n",
   "Dr. Chauhan's diplomas were Ph.D. diplomas.\n",
   'It was enjoyable.\n'],
  ["I stopped by Dr. Chauhan's home.\n",
   'Dr. Chauhan was displaying his framed degrees.\n',
   'Dr. Chauhan has a B.S. degree.\n',
   'Dr. Chauhan has an M.S. degree.\n',
   'Dr. Chauhan has a Ph.D. degree.\n',
   'The experience was delightful.\n']],
 [['I took a school trip.\n',
   'The school trip was to Washington, D.C.\n',
   'I went to see the White House.\n',
   'I was amazed by the artwork.\n',
   'The artwork was stunning.\n',
   'I was amazed by the individuals.\n',
   'The individuals were notable.\n',
 